In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install unrar
# !unrar x /content/drive/MyDrive/Images.rar

In [ ]:
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers, losses, metrics, optimizers
import cv2

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, LeakyReLU, Softmax
from tensorflow.keras.optimizers import SGD

# Load the Xception model
xception_model = load_model('/content/drive/MyDrive/xceptionmodel.h5')
# Freeze the layers in the Xception base model
for layer in xception_model.layers:
    layer.trainable = False

# Create a custom classification head
x = xception_model.output
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dense(256)(x)
x = LeakyReLU(alpha=0.4)(x)

# Change the output layer for binary classification with softmax
output = Dense(1, activation='sigmoid')(x)

# Create the custom model
model = Model(inputs=xception_model.input, outputs=output)

# Compile the model with the SGD optimizer and binary cross-entropy loss
sgd = SGD(learning_rate=0.3, momentum=0.9)  # You can adjust the learning rate and momentum as needed
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary to see the architecture
model.summary()


In [ ]:
k = load_model('/content/drive/MyDrive/MyFolder/myModel')

In [ ]:
k.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    '/content/Images/',
    batch_size=256,
    class_mode='categorical',
    target_size=(71,71),
    shuffle=True)

Found 75006 images belonging to 2 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10)

Epoch 1/10
100/100 [==============================] - 891s 9s/step - loss: 0.7051 - accuracy: 0.5000
Epoch 2/10
100/100 [==============================] - 876s 9s/step - loss: 0.6933 - accuracy: 0.5000
Epoch 3/10
100/100 [==============================] - 864s 9s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 4/10
100/100 [==============================] - 871s 9s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 5/10
100/100 [==============================] - 868s 9s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 6/10
100/100 [==============================] - 876s 9s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 7/10
100/100 [==============================] - 868s 9s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 8/10
100/100 [==============================] - 873s 9s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 9/10
100/100 [==============================] - 864s 9s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 10/10
100/100 [==============================] - 897s 9s/step - loss: 0.6931 - accura

In [ ]:
model.save('/content/drive/MyDrive/MyFolder/myModel', save_format="h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
# from keras.applications.vgg16 import preprocess_input

#load the image
my_image = load_img('Images/Train/Real/000_20_1.png', target_size=(299, 299))

#preprocess the image
my_image = img_to_array(my_image)
my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
my_image = preprocess_input(my_image)

#make the prediction
prediction = model.predict(my_image)
if prediction[0][0] > 0.5:
    print("Real")
else:
    print("Fake")

1/1 [==============================] - 0s 199ms/step
Real


In [ ]:
import os
import shutil
import numpy as np
real = np.random.choice(os.listdir("Images/Train/Real/"),1000,replace=False)
fake = np.random.choice(os.listdir("Images/Train/Fake/"),1000,replace=False)

In [ ]:
import numpy as np
for i in real:
    shutil.copy("Images/Train/Real/"+i,'Images/Test/Real/'+i)
for i in fake:
    shutil.copy("Images/Train/Fake/"+i,'Images/Test/Fake/'+i)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator( preprocessing_function=preprocess_input,)
test_generator = test_datagen.flow_from_directory(
    'Images/Test/',
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

Found 2000 images belonging to 2 classes.


In [ ]:
predictions = model.predict(test_generator)

63/63 [==============================] - 224s 4s/step


In [ ]:
pred_label = []

for i in predictions:
    if i>0.5:
        pred_label.append(1)
    else:
        pred_label.append(0)

In [ ]:
metrics.Accuracy()(pred_label,test_generator.labels)

<tf.Tensor: shape=(), dtype=float32, numpy=0.517>

In [ ]:
import cv2, numpy as np
import dlib
import matplotlib.pyplot as plt
face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [ ]:
def video_classifiaction(video_path, model):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        obj = face_detector(grey)
        if len(obj)!=0:
            face = obj[0]
            my_image = frame[face.top():face.bottom(),face.left():face.right()]
            my_image = cv2.resize(my_image, (299,299), interpolation = cv2.INTER_AREA)
            my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
            my_image = preprocess_input(my_image)

            #make the prediction
            prediction = model.predict(my_image)
            if prediction[0][0] > 0.5:
                print("Real")
            else:
                print("Fake")

In [ ]:
video_classifiaction('original_sequences/youtube/c23/videos/007.mp4',k)

1/1 [==============================] - 0s 252ms/step
Fake
1/1 [==============================] - 0s 259ms/step
Fake
1/1 [==============================] - 0s 217ms/step
Fake
1/1 [==============================] - 0s 229ms/step
Fake
1/1 [==============================] - 0s 189ms/step
Fake
1/1 [==============================] - 0s 227ms/step
Fake
1/1 [==============================] - 0s 174ms/step
Fake
1/1 [==============================] - 0s 223ms/step
Fake
1/1 [==============================] - 0s 186ms/step
Fake
1/1 [==============================] - 0s 185ms/step
Fake
1/1 [==============================] - 0s 231ms/step
Fake
1/1 [==============================] - 0s 221ms/step
Fake
1/1 [==============================] - 0s 182ms/step
Fake
1/1 [==============================] - 0s 147ms/step
Fake
1/1 [==============================] - 0s 174ms/step
Fake
1/1 [==============================] - 0s 178ms/step
Fake
1/1 [==============================] - 0s 180ms/step
Fake
1/1 [=========

In [ ]:
75000/6000

12.5

In [ ]:
75000/32

2343.75